# Doing without `class`

A talk for [CamPUG](https://www.meetup.com/CamPUG/) by Tibs (they/he)

Sources are at https://github.com/tibs/doing-without-class

Licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/)

## Abstract

When I named this talk, I realised that the name was ambiguous.

I could have changed the name, but instead I figured I'd do both talks.

* Is it OK to write Python code without classes?
* Can we create Python classes without using the `class` keyword

## Part 1: Is it OK to write Python code without classes?

**tldr;** Yes

Ideas:
    
* Write the simplest thing that will get the job done
* Python is deliberately a multi-paradigm language

[The seven programming ur-languages](https://madhadron.com/posts/seven_languages.html)

Broadly: 
    ALGOL,
    Lisp,
    ML,
    Self,
    Forth,
    APL,
    Prolog


Python clearly fits well in the "ALGOL" family.

[Stop Writing Classes](https://pyvideo.org/pycon-us-2012/stop-writing-classes.html)

Talk from PyCon US 2012, "Classes are great but they are also overused. This talk will describe examples of
class overuse taken from real world code and refactor the unnecessary classes, exceptions, and modules out of them."

There's nothing wrong with:

1. Write some simple linear code
2. Ooh - functions would make this easier - refactor to use functions
3. Ooh - maybe I need a class - refactor to use it

#### Some history

...from a C programmer

Back in the late 1980s, we were writing C code that created a `context` datastructure, which we would pass to functions.

This would contain the bundle of data items that most functions needed most of the time.

We would put it as the first argument to those functions.

Later on, we had the idea to add pointers to common functions to those contexts.

So we'd typically have `print` and `compare` function pointers, and we'd attach the appropriate function *for that particular type of context*.

Which we'd do in an `init` function.

Yes, we too invented some of the basics of OO.

Just like lots of people, I'm sure.

What's the point?

Well, sometimes it's OK to pass a context value around, without turning things "inside out" and creating a class.

And, to be honest, if there are only a few values in that context, it may not even be worth creating it. Just pass the values.

#### Some history

...from a Fortan programmer

Before we used C, we used Fortran. It was somewhere between FORTRAN IV and Fortran 77.

So a lot of the communication between functions and subroutines was done with `COMMON` blocks - essentially named global storage.

We managed.

Personally, I am a library writer at heart, so am unreasonably prejudiced against using Python `global`s.

But if you're writing a simple *program*

* and all of the functions in that program use a few shared values, 
* and especially if few (if any) of them modify those values

then globals can make sense.

#### In summary

* It's OK to keep things simple
* It's OK not to use classes if they're not needed
* It's OK to pass around `context` values
* It's OK (in a program, not in a library!) to use `global` values

**But** be prepared to refactor when it gets hard to understand.

## Part 2: Can we create Python classes without using the `class` keyword

(and some other stuff)

### Looking at a simple class

In [1]:
class Example:
    """A simple example class"""
    a = 3
    def f(self, p):
        """A function that takes a single argument"""
        return f'The parameters were {self} and {p}'

Which we can use in the expected manner

In [2]:
e = Example()
print(f'Class {Example}')
print(f'Instance {e}')
print(f'Value    {e.a}')
print(f'Method   {e.f}')
print(f'Calling the method with 3 {repr(e.f(3))}')

Class <class '__main__.Example'>
Instance <__main__.Example object at 0x1033ed070>
Value    3
Method   <bound method Example.f of <__main__.Example object at 0x1033ed070>>
Calling the method with 3 'The parameters were <__main__.Example object at 0x1033ed070> and 3'


### Using values

We can update the `a` on the class and see it also change on the instance

In [3]:
Example.a = 4
print(Example.a)
print(e.a)

4
4


But if we update the `a` on the instance

In [4]:
print(e.a)
e.a += 1
print(e.a)

4
5


It doesn't change `a` on the class

In [5]:
print(Example.a)

4


And now changing it on the class won't touch the value on the instance

In [6]:
Example.a = 99
print(f'Class    "a" {Example.a}')
print(f'Instance "a" {e.a}')

Class    "a" 99
Instance "a" 5


If we add a new value to the class, the instance will get it

In [7]:
Example.b = 12
print(f'Class    "b" {Example.b}')
print(f'Instance "b" {e.b}')

Class    "b" 12
Instance "b" 12


But not the other way round

In [8]:
e.c = -1
print(f'Instance has "c" {hasattr(e, "c")}')
print(f'Instance "c" {e.c}')
print(f'Class has "c" {hasattr(Example, "c")}')
print(f'Class    "c" {Example.c}')

Instance has "c" True
Instance "c" -1
Class has "c" False


AttributeError: type object 'Example' has no attribute 'c'

### Using methods

We can call the method on the instance:

In [10]:
e.f(4)

'The parameters were <__main__.Example object at 0x1033ed070> and 4'

Or via the class, although then we need to pass in the instance (`self`) explicitly

In [11]:
Example.f(e, 4)

'The parameters were <__main__.Example object at 0x1033ed070> and 4'

and there's nothing special about `self`

In [12]:
Example.f('not an instance', 4)

'The parameters were not an instance and 4'

So we shouldn't be surprised that on the class it's a function, but on the instance it's a method, which gets passed the instance as its first argument

In [13]:
print(f'Example.f is a {type(Example.f)}')
print(f'e.f is a {type(e.f)}')

Example.f is a <class 'function'>
e.f is a <class 'method'>


**Aside** In Python, all methods are functions, and there's nothing special about the name `self`.

In [14]:
def double(self, x):
    return x + x

Example.double = double
print(f'Class has "double" {hasattr(Example, "double")}')
print(f'Instance has "double" {hasattr(e, "double")}')
print(f'Example.double is a {type(Example.double)}')
print(f'e.double is a {type(e.double)}')
print(f'e.double(3) is {e.double(3)}')

Class has "double" True
Instance has "double" True
Example.double is a <class 'function'>
e.double is a <class 'method'>
e.double(3) is 6


Contrariwise, in Ruby, all functions are methods:

```ruby
irb(main):005:1* def fn(a)
irb(main):006:1*   puts "self is #{self} #{self.inspect}"
irb(main):007:0> end
=> :fn
irb(main):008:0> fn(1)
self is main main
=> nil
```

It seems natural to me that in Python we have to specify the `self` argument explicitly, as it's not special *to the function*.

However, in Ruby, as all functions are methods, there's always a parent class, and always a `self`, so there's no need to put it in the argument list.

**End of aside**

### Can we define a class without `class`?

Can we create a class without using the `class` keyword?

We've already used the `type` callable:

In [16]:
type(Example)

type

(Although I would have guessed a class would be of type "class")

and of course

In [19]:
print(type(1))
print(type('2'))

<class 'int'>
<class 'str'>


While `type` looks like a function, interestingly it isn't

If we ask for `help(type)`, we see:
```
Help on class type in module builtins:

class type(object)
 |  type(object_or_name, bases, dict)
 |  type(object) -> the object's type
 |  type(name, bases, dict) -> a new type
 ...
```

We've been using the first of those, `type(object) -> the object's type`.

and now we want the second, `type(name, bases, dict) -> a new type`

In [24]:
EmptyClass = type('EmptyClass', (), {})

In [25]:
print(type(EmptyClass))
print(repr(EmptyClass))

<class 'type'>
<class '__main__.EmptyClass'>


In [27]:
ec = EmptyClass()
print(type(ec))
print(repr(ec))

<class '__main__.EmptyClass'>


Let's define a convenient "method"

In [11]:
def function_f(self, p):
    """A function we shall use as a method that takes a single argument"""
    return f'The parameters were {self} and {p}'

And use it to build a simple class

In [12]:
ByHand = type('ByHand', (), {'f': function_f, 'a': 3})

In [30]:
bh = ByHand()
print(bh)
print(bh.a)
print(bh.f(2))

3
The parameters were <__main__.ByHand object at 0x1034d9a60> and 2


The obvious next thing to do is to make a function to make classes

In [35]:
from collections import ChainMap
def make_a_class(name, value_dict, method_dict):
    cls = type(name, (), dict(ChainMap(value_dict, method_dict)))
    return cls

In [36]:
C = make_a_class('ByHand', {'a': 3}, {'f': function_f})
print(f'Class {C!r}')
print(f'Class value a {C.a!r}')
print(f'Class function {C.f(None, "fred")}')

Class <class '__main__.ByHand'>
Class value a 3
Class function The parameters were None and fred


We can create an instance, just as we might expect

In [38]:
o = C()
print(f'Instance {o!r}')
print(f'Instance vaue a {o.a!r}')
print(f'Instance function {o.f("fred")}')

Instance <__main__.ByHand object at 0x1034a86a0>
Instance vaue a 3
Instance function The parameters were <__main__.ByHand object at 0x1034a86a0> and fred


### Can we construct an instance by hand?

Can we create an empty object and add things to it?

Our first guess might be to create an instance of the base class, `Object`

In [14]:
o = object()

but unfortunately, its not possible to add new values to instances of `Object`

In [15]:
o.a = 1

AttributeError: 'object' object has no attribute 'a'

So we still have to use `type` to get an empty mutable object

In [16]:
EmptyClass = type('EmptyClass', (), {})
eo = EmptyClass()
print(type(eo))

<class '__main__.EmptyClass'>


And we know we can do

In [17]:
eo.a = 1
print(eo.a)

1


In [18]:
eo.a = eo.a + 1
print(eo.a)

2


Can we add a function *to the object* and have it be a method?

In [19]:
def maybe_a_method(self, x):
    print(f'Maybe a method on {self} and {x}')
    
eo.f = maybe_a_method
print(eo.f(1))

TypeError: maybe_a_method() missing 1 required positional argument: 'x'

Unfortunately, adding a function as a value on an instance doesn't make a method

In [21]:
print(type(eo.f))

<class 'function'>


Normally, when we ask an instance for a method (`eo.f`), it gets looked up in the instance, isn't found there,
and is looked up in the class, which says
"I know what you're doing, that's a function you're looking up on me, so you must want a method back"

In [25]:
class NoMethods: pass
def just_a_function(self): return 'Aha!'
NoMethods.f = just_a_function
nm = NoMethods()

print(type(just_a_function))
print(type(NoMethods.f))
print(type(nm.f))
print(nm.f())

<class 'function'>
<class 'function'>
<class 'method'>
Aha!


But our empty object is an instance of an empty class, so that won't work.

Luckily there is a way:

In [27]:
eo.f = just_a_function.__get__(eo, EmptyClass)
print(type(eo.f))
print(eo.f())

<class 'method'>
Aha!


I don't propose to explain that (but am grateful to https://stackoverflow.com/a/46757134 for the example!).

If you want to learn more, then this is using the power of *descriptors*, which are at the heart of Python's attribute access

See the HOWTO at https://docs.python.org/3/howto/descriptor.html

There is also a more "understandable" way to do this.

We can create a `method` from our function

In [28]:
import types
eo.f = types.MethodType(just_a_function, eo)
print(type(eo.f))
print(eo.f())

<class 'method'>
Aha!


And we can create a function to do wrap this nicely

In [29]:
def pretend_instance(class_name, variable_dict, function_list):
    eo_class = type(class_name, (), variable_dict)
    eo = eo_class()
    for f in function_list:
        setattr(eo, f.__name__, types.MethodType(f, eo))
    return eo

In [30]:
x = pretend_instance('ClassName', {'var': 3}, [just_a_function])
print(type(x))
print(x.var)
print(x.just_a_function())

<class '__main__.ClassName'>
3
Aha!


### Can we create a function without using `def`?

Well, there is lambda

In [ ]:
lamb = lambda x: x + 1

lamb(2)

although

1. That's another keyword
2. It's very limited in what it can do

> An anonymous inline function consisting of a single expression which is evaluated when the function is called

There is also `types.FunctionType`, which is similar in idea to our use of `type` to create classes.

(unfortunately, it's signature is implementation specific and may even change between Python versions)

```python
>>> help(types.FunctionType)
Help on class function in module builtins:

class function(object)
 |  function(code, globals, name=None, argdefs=None, closure=None)
 |
 |  Create a function object.
 |
 |  code
 |    a code object
 |  globals
 |    the globals dictionary
 |  name
 |    a string that overrides the name from the code object
 |  argdefs
 |    a tuple that specifies the default argument values
 |  closure
 |    a tuple that supplies the bindings for free variables
 ```

We can get a code object from an existing function

In [ ]:
print(lamb.__code__)

In [ ]:
import types
lambish = types.FunctionType(lamb.__code__, globals())
print(lambish(1))

or we can create one with `compile`

In [ ]:
code = compile('print(4)', 'no-file', 'exec')
compiled_fn = types.FunctionType(code, globals())
print(compiled_fn())

But how did `lambish` know about its argument?

In [ ]:
print(dir(lamb.__code__))

The documentation for the `inspect` module tells us that `co_argcount` is the number of arguments, and `co_varnames` is a tuple of the names of the arguments and then the names of the local variables

In [ ]:
print(lamb.__code__.co_argcount)
print(lamb.__code__.co_varnames)

But that't not mutable

In [ ]:
lamb.__code__.co_varnames = ('x', 'y')

Let's try looking at the "inside" of lambish

In [ ]:
import dis
dis.dis(lambish)

In [ ]:
dis.dis(lamb.__code__)

However, there is `signature` (which needs a callable as its argument)

In [ ]:
print(inspect.signature(lambish))

and that actually returns an instance of class `Signature`

In [ ]:
print(type(inspect.signature(lambish)))

In [ ]:
import dis
dis.dis(fn)

In [ ]:
dis.dis(fn.__code__)

In [ ]:
dis.dis(bare_f)

In [ ]:
print(dir(fn))

In [ ]:
print(fn.__dict__)

In [ ]:
print(fn.__call__)

In [ ]:
print(fn.__call__())

In [ ]:
print(dir(fn.f))

In [ ]:
save = fn.__code__
fn.__code__ = bare_f.__code__
print(fn(None, 'fred'))
fn.__code__ = save
print(fn())

-----------------

In [ ]:
a = lambda s, x: f'Lambda over #{s} and #{x}'

In [ ]:
print(a(1,2))

In [ ]:
dis.dis(a)

In [ ]:
def lf(s,x):
    pass

lf.__code__ = a.__code__

print(lf(1,2))

In [ ]:
lt = type('Function?', (), {})
lt.__code__ = a.__code__

In [ ]:
lt(1,2)

In [ ]:
print(dir(lf))

In [ ]:
import inspect

In [ ]:
print(inspect.isfunction(fn))
print(inspect.isfunction(fn.f))
print(inspect.isfunction(x.just_a_function))
print()
print(inspect.ismethod(fn))
print(inspect.ismethod(x.just_a_function))

In [9]:
print(inspect.isclass(Example))
print(inspect.isclass(eo_class))

NameError: name 'inspect' is not defined

There's no point in asking if something is an instance, because everything is...

In [ ]:
inspect.signature(just_a_function)

In [ ]:
print(inspect.signature(bare_f))
print(inspect.getsource(bare_f))